<a href="https://colab.research.google.com/github/HoodBlack/CNN_SKKU/blob/main/PreTrainedViT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pretrained ViT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.0 MB/s eta 0:00:00


In [4]:
import os
import shutil
import numpy as np
import pandas as pd
import timm
import matplotlib.pyplot as plt

In [5]:
pip install torch_optimizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.5 MB/s eta 0:00:00


In [6]:
import torch
from PIL import Image
from skimage import io
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision import models

In [7]:
import json
import csv

def json_to_csv(folder_path, output_file):
    # CSV 파일을 작성하기 위한 파일 핸들러를 엽니다.
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)

        writer.writerow(['file_name','grade'])
        # 폴더 내의 모든 파일에 대해 반복합니다.
        for filename in os.listdir(folder_path):
            if filename.endswith('.json'):  # JSON 파일인지 확인합니다.
                file_path = os.path.join(folder_path, filename)

                # JSON 파일을 읽어옵니다.
                with open(file_path, 'r') as json_file:
                    data = json.load(json_file)
                    file_name = data.get('label_info', {}).get('image', {}).get('file_name', '')
                    grade = data.get('label_info', {}).get('shapes', [{}])[0].get('grade', '')
                    writer.writerow([file_name, grade])


In [8]:
class meatTransform():
  def __init__(self, resize):
    # ToTensor() -> H x W x C => C x H x W ?
    self.data_transform={'train': transforms.Compose([transforms.Resize([resize, resize]),
                                                      transforms.RandomHorizontalFlip(p=0.4),
                                                      transforms.RandomVerticalFlip(p=0.4),
                                                      transforms.ToTensor()]),
                         'val': transforms.Compose([transforms.Resize([resize, resize]),
                                                    transforms.RandomHorizontalFlip(p=0.4),
                                                    transforms.RandomVerticalFlip(p=0.4),
                                                    transforms.ToTensor()])}

  def __call__(self,img, phase='train'):
    return self.data_transform[phase](img)



In [9]:
class MeatDataset(Dataset):

  def __init__(self, csv_file, root_dir, phase,transform =None):
    self.meat_data = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.phase = phase
    self.transform = transform

  def __len__(self):
    return len(self.meat_data)

  def __getitem__(self, idx):
    img_name = os.path.join(self.root_dir, self.meat_data.iloc[idx,0])
    img = io.imread(img_name)
    transfromed_img = self.transform(img, self.phase)
    label = self.meat_data.iloc[idx,1]

    return transfromed_img, label


In [10]:
size = 448

# 사용 예시
folder_path_train = '/content/drive/MyDrive/Meat_Data/Training.zip (Unzipped Files)/Training/original_label_mix'  # JSON 파일들이 있는 폴더 경로로 변경해야 합니다.
output_file_train = '/content/drive/MyDrive/Meat_Data/Training.zip (Unzipped Files)/Training/origianl_cow_mix/meat_train_annot.csv' # 결과를 저장할 CSV 파일 경로로 변경해야 합니다.

folder_path_valid='/content/drive/MyDrive/Meat_Data/Training.zip (Unzipped Files)/Training/origianl_cow_mix'
output_file_valid='/content/drive/MyDrive/Meat_Data/Training.zip (Unzipped Files)/Validation/original_cow_mix/meat_valid_annot.csv'

json_to_csv(folder_path_train, output_file_train)

json_to_csv(folder_path_valid, output_file_valid)


In [12]:
data=pd.read_csv(output_file_train)

In [1]:
data['grade'].describe()

NameError: ignored

In [ ]:
train_csv = '/content/drive/MyDrive/meat_data/원천데이터/1. 소도제/1.segmentation/1/meat_annot.csv'
train_root = '/content/drive/MyDrive/meat_data/원천데이터/1. 소도제/1.segmentation/1'
valid_csv =
valid_root=

train_dataset = MeatDataset(csv_file=train_csv, root_dir=train_root, phase='train',transform=meatTransform(size))
valid_dataset = MeatDataset(csv_file=valid_csv, root_dir = valid_root, pahse = 'valid', transform = meatTransform(size))


In [ ]:
batch_size =32

train_Dataloader  = DataLoader(train_dataset, batch_size = batch_size, shuffle =True)

val_Dataloader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)

dataLoaders_dict = {"train" : train_Dataloader, "val": val_Dataloader}

# Models

In [19]:
pretrained_model_list = timm.list_models(pretrained=True)

print(pretrained_model_list)

['bat_resnext26ts.ch_in1k', 'beit_base_patch16_224.in22k_ft_in22k', 'beit_base_patch16_224.in22k_ft_in22k_in1k', 'beit_base_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_224.in22k_ft_in22k', 'beit_large_patch16_224.in22k_ft_in22k_in1k', 'beit_large_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_512.in22k_ft_in22k_in1k', 'beitv2_base_patch16_224.in1k_ft_in1k', 'beitv2_base_patch16_224.in1k_ft_in22k', 'beitv2_base_patch16_224.in1k_ft_in22k_in1k', 'beitv2_large_patch16_224.in1k_ft_in1k', 'beitv2_large_patch16_224.in1k_ft_in22k', 'beitv2_large_patch16_224.in1k_ft_in22k_in1k', 'botnet26t_256.c1_in1k', 'caformer_b36.sail_in1k', 'caformer_b36.sail_in1k_384', 'caformer_b36.sail_in22k', 'caformer_b36.sail_in22k_ft_in1k', 'caformer_b36.sail_in22k_ft_in1k_384', 'caformer_m36.sail_in1k', 'caformer_m36.sail_in1k_384', 'caformer_m36.sail_in22k', 'caformer_m36.sail_in22k_ft_in1k', 'caformer_m36.sail_in22k_ft_in1k_384', 'caformer_s18.sail_in1k', 'caformer_s18.sail_in1k_384', 'caformer_s

num_classes =0?

* https://huggingface.co/timm/vit_base_patch32_clip_448.laion2b_ft_in12k_in1k



In [20]:
num_classes = 2
model = timm.create_model('vit_base_patch32_clip_448',pretrained=True,num_classes = num_classes)

In [27]:
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler

citerion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

epochs = 10
for epoch in range(epochs):
  print("{}/{} epoch running now".format(epoch, epochs - 1))
  for phase in ['train', 'val']:
    if phase == 'train':
      model.train()
    else:
      model.eval()
      running_loss = 0.0
      running_corrects = 0

      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        # requires_grad = True, when Training
        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          if phase == 'train':
            # 손실 계산 (각 Tensor의 gradient 계산)
            loss.backward()
            # 계산된 손실을 optimizer의 계산에 따라 weight 조정
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects = torch.sum(preds == labels.data)
      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double() / dataset_sizes[phase]

      print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        # 모델 저장
      if phase == 'val' and epoch_acc > best_acc :
        best_acc = epoch_acc
        torch.save('./best_model.pth')

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)